**Import packages**



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Input, Dense, Reshape, Conv2DTranspose,\
   Activation, BatchNormalization, ReLU, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

**Load data**

In [ ]:
Train_image_path ='../cifar100-dataset'
Test_image_path ='../set5-dataset'

**Add gaussian noise to training and testing data**

In [ ]:
noise_factor = 0.1
train_data_noisy = train_data_clean + noise_factor * numpy.random.normal(loc=0.0, scale=0.1, size=(32, 32, 3))
train_data_noisy = numpy.clip(train_data_noisy, 0., 1.)
test_data_noisy = test_data_clean + noise_factor * numpy.random.normal(loc=0.0, scale=0.1, size=(32, 32, 3))
test_data_noisy = numpy.clip(test_data_noisy, 0., 1.)

**Visualize data**

In [ ]:
idx = 4
plt.subplot(1,2,1)
plt.imshow(train_data_clean[idx])
plt.title('Original image')
plt.subplot(1,2,2)
plt.imshow(train_data_noisy[idx])
plt.title('Image with noise')
plt.show()

**Create model**

In [ ]:
def conv_block(x, filters, kernel_size, strides=2):
   x = Conv2D(filters=filters,
              kernel_size=kernel_size,
              strides=strides,
              padding='same')(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   return x
def deconv_block(x, filters, kernel_size):
   x = Conv2DTranspose(filters=filters,
                       kernel_size=kernel_size,
                       strides=2,
                       padding='same')(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   return x

In [ ]:
import time
tic=time.clock()
def denoising_autoencoder():
   dae_inputs = Input(shape=(32, 32, 3), name='dae_input')
   conv_block1 = conv_block(dae_inputs, 32, 3)
   conv_block2 = conv_block(conv_block1, 64, 3)
   conv_block3 = conv_block(conv_block2, 128, 3)
   conv_block4 = conv_block(conv_block3, 256, 3)
   conv_block5 = conv_block(conv_block4, 256, 3, 1)
   deconv_block1 = deconv_block(conv_block5, 256, 3)
   merge1 = Concatenate()([deconv_block1, conv_block3])
   deconv_block2 = deconv_block(merge1, 128, 3)
   merge2 = Concatenate()([deconv_block2, conv_block2])
   deconv_block3 = deconv_block(merge2, 64, 3)
   merge3 = Concatenate()([deconv_block3, conv_block1])
   deconv_block4 = deconv_block(merge3, 32, 3)
   final_deconv = Conv2DTranspose(filters=3,
                       kernel_size=3,
                       padding='same')(deconv_block4)
   dae_outputs = Activation('sigmoid', name='dae_output')(final_deconv) 
   return Model(dae_inputs, dae_outputs, name='dae')
dae=denoising_autoencoder()
dae.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
checkpoint = ModelCheckpoint('best_model.h5', verbose=1, save_best_only=True, save_weights_only=True)
dae.fit(train_data_noisy,
       train_data_clean,
       validation_data=(test_data_noisy, test_data_clean),
       epochs=40,
       batch_size=128,
       callbacks=[checkpoint])
toc=time.clock()

In [ ]:
toc-tic

In [ ]:
dae.load_weights('best_model.h5')
test_data_denoised = dae.predict(test_data_noisy)

In [ ]:
dae.summary()

**Visualize results**

In [ ]:
plt.figure(figsize=(20, 20))
print("Original Images ")
for i in range(0,5,1):
    plt.subplot(2, 10, i+1)
    plt.imshow(test_data_clean[i]) 
plt.savefig("Original Images.png")
plt.show()
plt.figure(figsize=(20, 20))
print("Noisy Images")
for i in range(0,5,1):
    plt.subplot(2, 10, i+1)
    plt.imshow(test_data_noisy[i]) 
plt.savefig("Noisy Images.png")
plt.show()
plt.figure(figsize=(20, 20))
print("Denoised Images")
for i in range(0,5,1):
    plt.subplot(2, 10, i+1)
    plt.imshow(test_data_denoised[i]) 
plt.savefig("Denoised Images.png")
plt.show()